In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
import boto3
import re
from datetime import datetime

import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer
from sagemaker.debugger import rule_configs, Rule, DebuggerHookConfig
from sagemaker.model_monitor import DataCaptureConfig, DatasetFormat, DefaultModelMonitor
from sagemaker.s3 import S3Uploader, S3Downloader

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

%cd /root/predicting-coronavirus
#%pip install mpu
import src.data_import as di
import src.data_tools as dt

%matplotlib inline
pd.set_option('display.max_columns', 500)  
pd.set_option('display.max_rows', 500)   

/root/predicting-coronavirus


In [4]:
sess = boto3.Session()
sm = sess.client('sagemaker')
role = sagemaker.get_execution_role()

In [11]:
account_id = sess.client('sts', region_name=sess.region_name).get_caller_identity()["Account"]
bucket = 'sagemaker-studio-{}-{}'.format(sess.region_name, account_id)
prefix = 'capstone2'

try:
    if sess.region_name == "us-east-1":
        sess.client('s3').create_bucket(Bucket=bucket)
    else:
        sess.client('s3').create_bucket(Bucket=bucket, 
                                        CreateBucketConfiguration={'LocationConstraint': sess.region_name})
except Exception as e:
    print("Looks like you already have a bucket of this name. That's good. Uploading the data files...")

# Return the URLs of the uploaded file, so they can be reviewed or used elsewhere
s3url = S3Uploader.upload('data/train3k.csv', 's3://{}/{}/{}'.format(bucket, prefix,'merged'))
s3url = S3Uploader.upload('data/val3k.csv', 's3://{}/{}/{}'.format(bucket, prefix,'merged'))
print(s3url)


s3://sagemaker-studio-us-east-1-752222400982/capstone2/merged/val3k.csv


In [12]:
from sagemaker.amazon.amazon_estimator import get_image_uri
docker_image_name = get_image_uri(boto3.Session().region_name, 'xgboost', repo_version='1.0-1')

In [13]:
s3_input_train = sagemaker.s3_input(s3_data='s3://sagemaker-studio-us-east-1-752222400982/capstone2/merged/train2.csv', content_type='csv')
s3_input_val = sagemaker.s3_input(s3_data='s3://sagemaker-studio-us-east-1-752222400982/capstone2/merged/val2.csv', content_type='csv')


In [14]:
smsess = sagemaker.session.Session()

create_date = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
coronavirus_xgb_experiment = Experiment.create(experiment_name="capstone2-xgb-{}".format(create_date), 
                                              description="Using xgboost to predict coronavirus cases", 
                                              sagemaker_boto_client=boto3.client('sagemaker'))

In [15]:
from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner

trial = Trial.create(trial_name="algorithm-mode-trial-{}".format(strftime("%Y-%m-%d-%H-%M-%S", gmtime())), 
                     experiment_name=coronavirus_xgb_experiment.experiment_name,
                     sagemaker_boto_client=boto3.client('sagemaker'))

xgb = sagemaker.estimator.Estimator(image_name=docker_image_name,
                                    role=role,
                                    train_instance_count=1, 
                                    train_use_spot_instances=True,
                                    train_max_run=300,
                                    train_max_wait=600,
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    base_job_name="xgboost-coronavirus",
                                    sagemaker_session=smsess)

xgb.set_hyperparameters(num_round=60, 
                        objective='reg:squarederror',
                        verbosity=1,
                        eta=0.1,
                        gamma=4)

hp_ranges = {
    "max_depth":IntegerParameter(4, 12),
    "subsample":ContinuousParameter(0.4, 0.8),
#    "eta":ContinuousParameter(0.03,0.2),
    "min_child_weight":IntegerParameter(1, 8),
#    "alpha":ContinuousParameter(0, 2)
}

tuner = HyperparameterTuner(
    xgb,
    'validation:rmse',
    hp_ranges,
    objective_type='Minimize',
    max_jobs=200,
    max_parallel_jobs=10
)

tuner.fit({'train': s3_input_train,
         'validation': s3_input_val}, 
                experiment_config={
                    "ExperimentName": coronavirus_xgb_experiment.experiment_name, 
                    "TrialName": trial.trial_name,
                    "TrialComponentDisplayName": "HPO Job"
                })



INFO:root:_TuningJob.start_new!!!
INFO:sagemaker:Creating hyperparameter tuning job with name: sagemaker-xgboost-200514-2147


In [ ]:
from sagemaker.analytics import HyperparameterTuningJobAnalytics

exp = HyperparameterTuningJobAnalytics(
    sagemaker_session=smsess,
    hyperparameter_tuning_job_name=tuner.latest_tuning_job.name)
df = exp.dataframe()
df.sort_values(by='FinalObjectiveValue')